# Tutorial MPC on Fuel Cell

## Imports

In [1]:
import os, sys
import numpy as np

from bokeh.io import output_notebook, show
output_notebook()

from neural_horizon_acados.mpc import * 
from neural_horizon_acados.plotting import *

from mpc import *
from model.model_src.ode_modelling.ode_model import get_model
from model.model_src.model_suh.model_suh import get_validation_input, model


Loading BokehJS ...

## Paths

In [2]:
RESULTS_DIR = os.path.abspath('Test_Results')
MPC_RESULTS_DIR = os.path.join(RESULTS_DIR, 'AMPC_results')

In [3]:
print(RESULTS_DIR)

/home/argo/work/scripts/acadosmpc/examples/fuel_cell/Test_Results


## General setup

In [4]:
mpc_param = SuH_MPC_Param(
    N_MPC=15,
    scale_model=True,
    acados_name='SQP_FCHPIPM',
    acados_settings=dict(
        nlp_solver_type = 'SQP',
        qp_solver = 'FULL_CONDENSING_HPIPM',
        integrator_type = 'ERK',
        hessian_approx = 'GAUSS_NEWTON',
        regularize_method = 'PROJECT_REDUC_HESS',
        sim_method_num_stages = 2,
        sim_method_num_steps = 4,
        use_x_guess = True,
    )
)
uref, _ = get_validation_input(mpc_param.Ts)
yref = np.vstack((uref[1], np.full_like(uref[1], mpc_param.yref[1])))

mpc_param.yref = yref

## acados MPC trajectory generation

In [5]:
results = SuH_AMPC.trajectory_and_save(
    mpc_param, 
    lambda x: get_model(model, x), 
    MPC_RESULTS_DIR, 
    always_overwrite=True
)

print(f'Cost: {results.Cost:.3f}')

15M_0v:   0%|          | 0/1200 [00:00<?, ?it/s]

Cost: 19.362


In [6]:
results = SuH_MPC_data.load(os.path.join(MPC_RESULTS_DIR, 'AMPC_results_SQP_FCHPIPM_15M_0v.ph'))

use_latex_style = False
figure_size = get_figure_size(fraction=1.0, ratio=5.) if use_latex_style else (1200, 200)

p_res = plot_MPC_results(
    [results],
    # group_by=lambda x: ,
    # plot_mpc_trajectories=True, 
    # xbnd=3, 
    activate_hover=True,
    thickness=[3 for _ in range(3)],
    dash=['solid' for _ in range(3)],
    # solver_time_scale='linear',
    additional_plots=['lambda_O2', 'Iterations', 'Prep_Time', 'Fb_Time'],
    additional_plots_options={
        'lambda_O2': {'y_axis_label': r'$$\lambda_{O2}$$'},
        'Iterations': {'y_axis_label': r'$$i_{\mathrm{sol}}$$'}, 
        'Prep_Time': {'y_axis_label': r'$$t_{\mathrm{prep}} \, [s]$$'},
        'Fb_Time': {'y_axis_label': r'$$t_{\mathrm{fb}} \, [s]$$'}, 
    },
    width=figure_size[0],
    height=figure_size[1],
    latex_style=use_latex_style,
    # ybnds=[[0.108e5, 0.112e5], [0.75e5, 0.85e5], [52e2, 68e2], [14e4, 16e4], [50, 250], [129.8, 130.1]],
    # ybnds=[[0.98e4, 1.3e4], [6e4, 9e4], [4e3, 1e4], [1.3e5, 1.8e5], [50, 250], [129.5, 130.5]]
    ybnds=[[0.5e4, 3e4], [2e4, 2e5], [4e3, 1.2e4], [0.8e5, 3.2e5], [10, 260], [80, 350]]
)
show(p_res)